In [ ]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [ ]:
#  Add the clean covid function that takes in the argument, "covid".
def clean_covid_data(covid):
    covid = dict(covid) #create a non-destructive copy

# merge column names
    def change_column_name(old_name, new_name):
        if old_covid in covid:
            covid[new_name] = covid.pop(old_name)
    change_column_name('Fips', 'FipCode')
    change_column_name('metrics.testPositivityRatio', 'PositivityRatio')
    change_column_name('metrics.caseDensity', 'Density')
    change_column_name('metrics.infectionRatio', 'InfectionRatio')
    change_column_name('metrics.infectionRatioCI90', 'InfectionRateCI90')
    change_column_name('metrics.icuHeadroomRatio', 'IcuHeadRoomRatio')
    change_column_name('metrics.icuCapacityRatio', 'IcuCapacityRatio')
    change_column_name('riskLevels.overall', 'RiskLevelOverall')
    change_column_name('riskLevels.testPositivityRatio', 'RiskLevelPositivityRatio')
    change_column_name('riskLevels.caseDensity', 'RiskLevelCaseDensity')
    change_column_name('riskLevels.contactTracerCapacityRatio', 'RiskLevelContactTracerCapacityRatio')
    change_column_name('riskLevels.infectionRate', 'RiskLevelInfectionRate')
    change_column_name('riskLevels.icuHeadroomRatio', 'RiskLevelICUHeadRoomRatio')
    change_column_name('riskLevels.icuCapacityRatio', 'RiskLevelsICUCapacityRatio')
    change_column_name('Actuals.cases', 'ActualCases')
    change_column_name('Actuals.deaths', 'ActualDeaths')
    change_column_name('actuals.hospitalBeds.capacity', 'ActualHospitalBedCapacity')
    change_column_name('actuals.hospitalBeds.currentUsageTotal', 'ActualHospitalBedCurrentUsageTotal')
    change_column_name('actuals.hospitalBeds.currentUsageCovid', 'ActualHospitalBedCurrentUsageCovid')
    change_column_name('actuals.hospitalBeds.typicalUsageRate', 'ActualHospitalBedTypicalUsageRate')
    change_column_name('actuals.icuBeds.capacity', 'ActualIcuBedsCapacity')
    change_column_name('actuals.icuBeds.currentUsageTotal', 'ActualIcuBedCurrentUsageTotal')
    change_column_name('actuals.icuBeds.currentUsageCovid', 'ActualIcuBedCurrentUsageCovid')
    change_column_name('actuals.icuBeds.typicalUsageRate', 'ActualIcuBedTypicalUsageRate')
    change_column_name('actuals.newCases', 'ActualNewCases')
    change_column_name('actuals.vaccinationsInitiated', 'ActualVaccinationInitiated')
    change_column_name('actuals.vaccinationsCompleted', 'ActualVaccinationCompleted')
    change_column_name('metrics.vaccinationsInitiatedRatio', 'VaccinationInitaitedRatio')
    change_column_name('metrics.vaccinationsCompletedRatio', 'VaccinationCompletedRatio')
    change_column_name('actuals.newDeaths', 'ActualNewDeaths')
    change_column_name('actuals.vaccinesAdministered', 'ActualVaccinesAdministered')


    return covid

In [ ]:
# 1 Add the function that takes in one argument;
# covid data from apidocs.covidactnow.org

def extract_transform_load(covid, epa):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    # kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    # ratings = pd.read_csv(ratings_file, low_memory=False)

    # Open and read the Wikipedia data JSON file.
    with open(covid_file, mode='r') as file:
        covid_raw = json.load(file)


    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_covid = [clean_covid(covid) for covid in wiki_movies]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset="imdb_id", inplace=True)
        
    except Exception:
        print("An error occured while extracting imdb duplicates..." (Exception)) 

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    

        

    wiki_movies_df.drop('Box office', axis=1, inplace=True)
  
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df["Budget"].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

    # Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
     
    # 2. Clean the Kaggle metadata.
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    #for col in movies_df.columns:
    #    lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
    #    value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
    #    num_values = len(value_counts)
    #    if num_values == 1:
    #        print(col)

    movies_df['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts(dropna=False)
    movies_df['original_language'].value_counts(dropna=False)
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
            , axis=1)
        df.drop(columns=wiki_column, inplace=True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')

    # 7. Filter the movies DataFrame for specific columns.
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

    # 8. Rename the columns in the movies DataFrame.
    movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Editor(s)':'editors',
                  'Production company(s)': 'production_companies',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

    # 9. Transform and merge the ratings DataFrame.
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')
    
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
    return wiki_movies_df, movies_with_ratings_df, movies_df

In [ ]:
# Path to your file directory and variables for the three files.
file_dir = "/Users/joepedroza/Desktop/Data Visualization Class/Analysis Projects/UT-VIRT-DATA-GROUP-6-PROJECT/Resources"
# COVID data by state
covid_by_state_file = f'{file_dir}/covid_by_state.json'
# COVID data by state series
covid_by_state_series_file = f'{file_dir}/covid_by_state_series.json'
# COVID data by state for Texas
covid_by_texas_file= f'{file_dir}/TX.csv'
# COVID data by state by date series for Texas
covid_by_texas_file= f'{file_dir}/TX.timeseries.csv'

In [ ]:
# Set the variables equal to the extract_transform_load function.
covid_texas_file = extract_transform_load(covid_by_texas_file)